<center><img src="image.png" width=500></center>
<p>

The task is to automate the process of retrieving and cleaning data by building a data pipeline to pull data each month, in order to observe changes in the sales and capability of different energy types.

Two raw files; `electricity_sales.csv` and `electricity_capability_nested.json`. 
    
Below is a data dictionary for the `electricity_sales.csv` dataset, which data are transformed in.

| Field | Data Type |
| :---- | :-------: |
| period  | `str`        |
| stateid | `str` |
| stateDescription | `str` |
| sectorid | `str` |
| sectorName | `str` |
| price | `float` |
| price-units | `str` |

In [35]:
import pandas as pd
import json
import os

In [36]:
def extract_tabular_data(file_path: str):
    """Extract data from a tabular file_format, with pandas."""
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path)
        return df
    elif file_path.endswith(".parquet"):
        df = pd.read_csv(file_path)
        return df
    else:
        raise Exception("Warning: Invalid file extension. Please try with .csv or .parquet!")

In [37]:
def extract_json_data(file_path):
    """Extract and flatten data from a JSON file."""
    with open(file_path, 'r') as json_data:
        df_nested = json.load(json_data)
    df = pd.json_normalize(df_nested)
    return df

In [38]:
def transform_electricity_sales_data(raw_data: pd.DataFrame):
    """
    Transform electricity sales to find the total amount of electricity sold
    in the residential and transportation sectors.
    
    To transform the electricity sales data, you'll need to do the following:
    - Drop any records with NA values in the `price` column. Do this inplace.
    - Only keep records with a `sectorName` of "residential" or "transportation".
    - Create a `month` column using the first 4 characters of the values in `period`.
    - Create a `year` column using the last 2 characters of the values in `period`.
    - Return the transformed `DataFrame`, keeping only the columns `year`, `month`, `stateid`, `price` and `price-units`.
    """
    na_dropped = raw_data.dropna(subset=["price"])
    filtered = na_dropped[na_dropped["sectorName"].isin(["residential", "transportation"])]
    filtered.loc[:, "month"] = filtered["period"].str[5,7]
    filtered.loc[:, "year"] = filtered["period"].str[0,4]
    df = filtered[["year", "month", "stateid", "price", "price-units"]]
    return df

In [39]:
def load(dataframe: pd.DataFrame, file_path: str):
    """Load a DataFrame to a file in either CSV or Parquet format."""
    if file_path.endswith(".csv"):
        dataframe.to_csv(file_path)
    elif file_path.endswith(".parquet"):
        dataframe.to_csv(file_path)
    else:
        raise Exception("Warning: {file_path} is not a valid type. Please try again!_")

In [40]:
def validation(file_path):
    if os.path.exists(file_path):
        return f"{file_path} exists."
    else:
        raise Exception(f"{filepath} not found.")